# 📚 Workshop: RAG + LangChain + Streamlit

## ช่วงที่ 4: Deploy with Streamlit UI (2:30 – 3:30)

---

### 🎯 วัตถุประสงค์การเรียนรู้
- รู้จัก Streamlit framework
- สร้าง UI chatbot
- กล่องข้อความให้ผู้ใช้พิมพ์คำถาม
- แสดงผลลัพธ์จาก RAG
- เก็บ history การสนทนา
- แสดง context ที่ RAG ดึงมา

---


## 🌐 Streamlit Framework

### Streamlit คืออะไร?

**Streamlit** เป็น Python framework สำหรับสร้าง web applications ได้อย่างง่ายและเร็ว

### ข้อดีของ Streamlit:

1. **ง่ายต่อการใช้งาน** 🚀
   - เขียนแค่ Python code
   - ไม่ต้องรู้ HTML/CSS/JavaScript

2. **เร็วในการพัฒนา** ⚡
   - สร้าง UI ได้ในไม่กี่บรรทัด
   - Auto-refresh เมื่อแก้ไขโค้ด

3. **เหมาะสำหรับ Data Science** 📊
   - รองรับการแสดงข้อมูล
   - มี widgets สำหรับ input/output

4. **Deploy ได้ง่าย** 🌐
   - Deploy บน Streamlit Cloud
   - รองรับ Docker

### Streamlit Components ที่จะใช้:
- `st.title()` - หัวข้อ
- `st.text_input()` - ช่องกรอกข้อความ
- `st.button()` - ปุ่ม
- `st.chat_message()` - ข้อความแชท
- `st.session_state` - เก็บข้อมูล session


In [1]:
# Import Libraries สำหรับ Streamlit
import streamlit as st
import os
import sys
from pathlib import Path

# เพิ่ม path สำหรับ import modules
sys.path.append('..')

# Import LangChain components
from langchain.chains import RetrievalQA
from langchain.chains.conversational_retrieval.base import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# Import utilities
from dotenv import load_dotenv
import logging

# ตั้งค่า logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# โหลด environment variables
load_dotenv()

print("✅ Streamlit และ LangChain libraries imported successfully!")


/home/game/01_Projects/workshop_rag_rmutl/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Streamlit และ LangChain libraries imported successfully!


## 🏗️ สร้าง RAG Service Class

### RAG Service Class

เราจะสร้าง class ที่รวม RAG functionality ทั้งหมด:

1. **โหลด Vector Store**
2. **ตั้งค่า LLM**
3. **สร้าง Chains**
4. **ตอบคำถาม**


In [2]:
# สร้าง RAG Service Class
class RAGService:
    """RAG Service สำหรับ Streamlit"""
    
    def __init__(self):
        self.vectorstore = None
        self.llm = None
        self.qa_chain = None
        self.conversation_chain = None
        self.embeddings = None
        
    def load_vectorstore(self, vectorstore_path="../vectorstore"):
        """โหลด Vector Store"""
        try:
            # โหลด embeddings
            self.embeddings = HuggingFaceEmbeddings(
                model_name="all-MiniLM-L6-v2",
                cache_folder="../model_cache"
            )
            
            # โหลด FAISS vector store
            self.vectorstore = FAISS.load_local(
                vectorstore_path, 
                self.embeddings, 
                allow_dangerous_deserialization=True
            )
            
            print(f"✅ โหลด Vector Store สำเร็จ: {self.vectorstore.index.ntotal} documents")
            return True
            
        except Exception as e:
            print(f"❌ ไม่สามารถโหลด Vector Store: {e}")
            return False
    
    def setup_llm(self, model_name="llama-3.3-70b-versatile", temperature=0.1):
        """ตั้งค่า LLM"""
        try:
            api_key = os.getenv("GROQ_API_KEY")
            if not api_key:
                raise ValueError("GROQ_API_KEY ไม่พบใน environment variables")
            
            self.llm = ChatGroq(
                groq_api_key=api_key,
                model_name=model_name,
                temperature=temperature
            )
            
            print(f"✅ ตั้งค่า Groq LLM สำเร็จ: {model_name}")
            return True
            
        except Exception as e:
            print(f"❌ เกิดข้อผิดพลาด: {e}")
            return False
    
    def create_custom_prompt(self):
        """สร้าง Custom Prompt Template"""
        prompt_template = """
คุณเป็นผู้ช่วยที่เชี่ยวชาญเกี่ยวกับข้อมูลในเอกสาร จงตอบคำถามอย่างกระชับและถูกต้อง

ข้อมูลอ้างอิง:
{context}

คำถาม: {question}

คำตอบ:"""
        
        return PromptTemplate(
            template=prompt_template,
            input_variables=["context", "question"]
        )
    
    def setup_qa_chain(self, k=3):
        """ตั้งค่า QA Chain"""
        try:
            if not self.vectorstore or not self.llm:
                raise ValueError("ต้องมี vectorstore และ llm")
            
            # สร้าง retriever
            retriever = self.vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={"k": k}
            )
            
            # สร้าง custom prompt
            custom_prompt = self.create_custom_prompt()
            
            # สร้าง QA chain
            self.qa_chain = RetrievalQA.from_chain_type(
                llm=self.llm,
                chain_type="stuff",
                retriever=retriever,
                chain_type_kwargs={"prompt": custom_prompt},
                return_source_documents=True
            )
            
            print("✅ สร้าง QA Chain สำเร็จ")
            return True
            
        except Exception as e:
            print(f"❌ เกิดข้อผิดพลาด: {e}")
            return False
    
    def setup_conversation_chain(self, k=3):
        """ตั้งค่า Conversation Chain"""
        try:
            if not self.vectorstore or not self.llm:
                raise ValueError("ต้องมี vectorstore และ llm")
            
            # สร้าง memory
            memory = ConversationBufferMemory(
                memory_key="chat_history",
                return_messages=True
            )
            
            # สร้าง retriever
            retriever = self.vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={"k": k}
            )
            
            # สร้าง conversation chain
            self.conversation_chain = ConversationalRetrievalChain.from_llm(
                llm=self.llm,
                retriever=retriever,
                memory=memory,
                return_source_documents=True
            )
            
            print("✅ สร้าง Conversation Chain สำเร็จ")
            return True
            
        except Exception as e:
            print(f"❌ เกิดข้อผิดพลาด: {e}")
            return False
    
    def answer_question(self, question, use_conversation=False):
        """ตอบคำถาม"""
        try:
            if use_conversation and self.conversation_chain:
                result = self.conversation_chain({"question": question})
                answer = result["answer"]
                source_docs = result.get("source_documents", [])
            elif self.qa_chain:
                result = self.qa_chain({"query": question})
                answer = result["result"]
                source_docs = result.get("source_documents", [])
            else:
                raise ValueError("ไม่มี chain ให้ใช้งาน")
            
            # แปลง source documents
            sources = []
            for doc in source_docs:
                sources.append({
                    "content": doc.page_content,
                    "metadata": doc.metadata
                })
            
            return {
                "answer": answer,
                "sources": sources,
                "method": "conversation" if use_conversation else "qa"
            }
            
        except Exception as e:
            return {
                "answer": f"เกิดข้อผิดพลาด: {e}",
                "sources": [],
                "method": "error"
            }
    
    def initialize(self):
        """เริ่มต้นระบบทั้งหมด"""
        print("🚀 เริ่มต้น RAG Service...")
        
        # โหลด vectorstore
        if not self.load_vectorstore():
            return False
        
        # ตั้งค่า LLM
        if not self.setup_llm():
            return False
        
        # ตั้งค่า chains
        if not self.setup_qa_chain():
            return False
        
        if not self.setup_conversation_chain():
            return False
        
        print("✅ RAG Service เริ่มต้นสำเร็จ")
        return True

# สร้าง RAG Service instance
rag_service = RAGService()
print("📦 RAG Service Class สร้างสำเร็จ")


📦 RAG Service Class สร้างสำเร็จ


## 🎨 สร้าง Streamlit UI

### Streamlit App Structure:

1. **Page Configuration** - ตั้งค่าหน้า
2. **Sidebar** - การตั้งค่า
3. **Main Area** - Chat interface
4. **Chat History** - ประวัติการสนทนา
5. **Input Area** - ช่องกรอกคำถาม


In [3]:
# สร้าง Streamlit App
def create_streamlit_app():
    """สร้าง Streamlit App"""
    
    # Page configuration
    st.set_page_config(
        page_title="RAG Chatbot with LangChain",
        page_icon="🤖",
        layout="wide",
        initial_sidebar_state="expanded"
    )
    
    # CSS styling
    st.markdown("""
    <style>
    .main-header {
        font-size: 2.5rem;
        color: #1f77b4;
        text-align: center;
        margin-bottom: 2rem;
    }
    .chat-message {
        padding: 1rem;
        border-radius: 0.5rem;
        margin: 0.5rem 0;
    }
    .user-message {
        background-color: #e3f2fd;
        border-left: 4px solid #2196f3;
    }
    .assistant-message {
        background-color: #f3e5f5;
        border-left: 4px solid #9c27b0;
    }
    .source-info {
        background-color: #f5f5f5;
        padding: 0.5rem;
        border-radius: 0.25rem;
        margin: 0.5rem 0;
        font-size: 0.9rem;
    }
    </style>
    """, unsafe_allow_html=True)
    
    # Main header
    st.markdown('<h1 class="main-header">🤖 RAG Chatbot with LangChain + Streamlit</h1>', unsafe_allow_html=True)
    
    # Sidebar
    with st.sidebar:
        st.header("⚙️ การตั้งค่า")
        
        # RAG Mode
        rag_mode = st.selectbox(
            "โหมด RAG",
            ["Simple QA", "Conversational"],
            help="Simple QA: ตอบคำถามแต่ละข้อ, Conversational: จดจำการสนทนา"
        )
        
        # K value
        k_value = st.slider(
            "จำนวนเอกสารที่ค้นหา (k)",
            min_value=1,
            max_value=10,
            value=3,
            help="จำนวนเอกสารที่ใช้ในการตอบคำถาม"
        )
        
        # Temperature
        temperature = st.slider(
            "Temperature",
            min_value=0.0,
            max_value=1.0,
            value=0.1,
            step=0.1,
            help="ควบคุมความสร้างสรรค์ของคำตอบ"
        )
        
        # Clear chat button
        if st.button("🗑️ ล้างประวัติการสนทนา", use_container_width=True):
            st.session_state.messages = []
            st.rerun()
    
    # Initialize RAG Service
    @st.cache_resource
    def get_rag_service():
        """Get cached RAG service"""
        service = RAGService()
        if service.initialize():
            return service
        return None
    
    rag_service = get_rag_service()
    
    if rag_service is None:
        st.error("❌ ไม่สามารถเริ่มต้น RAG Service ได้ กรุณาตรวจสอบการตั้งค่า")
        st.info("💡 ตรวจสอบว่า:")
        st.info("- มีไฟล์ .env พร้อม GROQ_API_KEY")
        st.info("- มี Vector Store ในโฟลเดอร์ ../vectorstore")
        return
    
    # Initialize chat history
    if "messages" not in st.session_state:
        st.session_state.messages = []
    
    # Display chat history
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])
            
            # Show sources if available
            if message.get("sources"):
                with st.expander("📚 แหล่งข้อมูลอ้างอิง"):
                    for i, source in enumerate(message["sources"], 1):
                        st.markdown(f"**แหล่งที่ {i}:**")
                        st.markdown(f"เนื้อหา: {source['content'][:200]}...")
                        if source.get('metadata'):
                            st.markdown(f"ข้อมูลเพิ่มเติม: {source['metadata']}")
                        st.markdown("---")
    
    # Chat input
    if prompt := st.chat_input("พิมพ์คำถามของคุณ..."):
        # Add user message to chat history
        st.session_state.messages.append({"role": "user", "content": prompt})
        
        # Display user message
        with st.chat_message("user"):
            st.markdown(prompt)
        
        # Generate and display assistant response
        with st.chat_message("assistant"):
            with st.spinner("🤔 กำลังคิดด้วย RAG..."):
                try:
                    use_conversation = rag_mode == "Conversational"
                    response = rag_service.answer_question(prompt, use_conversation=use_conversation)
                    
                    # Display answer
                    st.markdown(response["answer"])
                    
                    # Show sources
                    if response.get("sources"):
                        with st.expander("📚 แหล่งข้อมูลอ้างอิง"):
                            for i, source in enumerate(response["sources"], 1):
                                st.markdown(f"**แหล่งที่ {i}:**")
                                st.markdown(f"เนื้อหา: {source['content'][:200]}...")
                                if source.get('metadata'):
                                    st.markdown(f"ข้อมูลเพิ่มเติม: {source['metadata']}")
                                st.markdown("---")
                    
                    # Add assistant response to chat history
                    st.session_state.messages.append({
                        "role": "assistant", 
                        "content": response["answer"],
                        "sources": response.get("sources", []),
                        "method": response.get("method", "unknown")
                    })
                    
                except Exception as e:
                    error_msg = f"❌ เกิดข้อผิดพลาด: {e}"
                    st.error(error_msg)
                    st.session_state.messages.append({"role": "assistant", "content": error_msg})
    
    # Quick questions
    st.markdown("---")
    st.subheader("❓ คำถามตัวอย่าง")
    
    # Categorized quick questions
    categories = {
        "🏛️ ประวัติศาสตร์": [
            "จังหวัดน่านมีประวัติศาสตร์อย่างไร?",
            "จังหวัดน่านเคยเป็นส่วนหนึ่งของอาณาจักรใดบ้าง?"
        ],
        "🏞️ สถานที่ท่องเที่ยว": [
            "จังหวัดน่านมีสถานที่ท่องเที่ยวอะไรบ้าง?",
            "จังหวัดน่านมีวัดสำคัญอะไรบ้าง?"
        ],
        "🍽️ อาหารและวัฒนธรรม": [
            "จังหวัดน่านมีอาหารพื้นเมืองอะไรบ้าง?",
            "จังหวัดน่านมีวัฒนธรรมอะไรที่น่าสนใจ?"
        ],
        "📊 ข้อมูลทั่วไป": [
            "จังหวัดน่านมีประชากรเท่าไหร่?",
            "จังหวัดน่านมีพื้นที่เท่าไหร่?"
        ]
    }
    
    for category, questions in categories.items():
        with st.expander(category):
            cols = st.columns(2)
            for i, question in enumerate(questions):
                with cols[i % 2]:
                    if st.button(question, use_container_width=True, key=f"quick_{category}_{i}"):
                        # Add user message to chat history
                        st.session_state.messages.append({"role": "user", "content": question})
                        
                        # Generate and add assistant response
                        with st.spinner("🤔 กำลังคิด..."):
                            try:
                                use_conversation = rag_mode == "Conversational"
                                response = rag_service.answer_question(question, use_conversation=use_conversation)
                                
                                # Add assistant response to chat history
                                st.session_state.messages.append({
                                    "role": "assistant", 
                                    "content": response["answer"],
                                    "sources": response.get("sources", []),
                                    "method": response.get("method", "unknown")
                                })
                                
                            except Exception as e:
                                error_msg = f"❌ เกิดข้อผิดพลาด: {e}"
                                st.session_state.messages.append({"role": "assistant", "content": error_msg})
                        
                        st.rerun()

# สร้าง Streamlit App
print("🎨 Streamlit App function สร้างสำเร็จ")
print("💡 ใช้คำสั่ง: streamlit run 04_streamlit_ui.py")


🎨 Streamlit App function สร้างสำเร็จ
💡 ใช้คำสั่ง: streamlit run 04_streamlit_ui.py


## 📄 สร้างไฟล์ Streamlit App

### สร้างไฟล์ `streamlit_app.py`


In [5]:
# สร้างไฟล์ streamlit_app.py
streamlit_app_code = '''# streamlit_app.py
import streamlit as st
import os
import sys
from pathlib import Path

# เพิ่ม path สำหรับ import modules
sys.path.append('.')

# Import LangChain components
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# Import utilities
from dotenv import load_dotenv
import logging

# ตั้งค่า logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# โหลด environment variables
load_dotenv()

class RAGService:
    """RAG Service สำหรับ Streamlit"""

    def __init__(self):
        self.vectorstore = None
        self.llm = None
        self.qa_chain = None
        self.embeddings = None

    def load_vectorstore(self, vectorstore_path="./vectorstore"):
        """โหลด Vector Store"""
        try:
            # โหลด embeddings
            self.embeddings = HuggingFaceEmbeddings(
                model_name="all-MiniLM-L6-v2",
                cache_folder="./model_cache"
            )

            # โหลด FAISS vector store
            self.vectorstore = FAISS.load_local(
                vectorstore_path, 
                self.embeddings, 
                allow_dangerous_deserialization=True
            )

            logger.info(f"โหลด Vector Store สำเร็จ: {self.vectorstore.index.ntotal} documents")
            return True

        except Exception as e:
            logger.error(f"ไม่สามารถโหลด Vector Store: {e}")
            return False

    def setup_llm(self, model_name="llama-3.3-70b-versatile", temperature=0.1):
        """ตั้งค่า LLM"""
        try:
            api_key = os.getenv("GROQ_API_KEY")
            if not api_key:
                raise ValueError("GROQ_API_KEY ไม่พบใน environment variables")

            self.llm = ChatGroq(
                groq_api_key=api_key,
                model_name=model_name,
                temperature=temperature
            )

            logger.info(f"ตั้งค่า Groq LLM สำเร็จ: {model_name}")
            return True

        except Exception as e:
            logger.error(f"เกิดข้อผิดพลาด: {e}")
            return False

    def create_custom_prompt(self):
        """สร้าง Custom Prompt Template"""
        prompt_template = """
คุณเป็นผู้ช่วยที่เชี่ยวชาญเกี่ยวกับข้อมูลในเอกสาร จงตอบคำถามอย่างกระชับและถูกต้อง

ข้อมูลอ้างอิง:
{context}

คำถาม: {question}

คำตอบ:"""

        return PromptTemplate(
            template=prompt_template,
            input_variables=["context", "question"]
        )

    def setup_qa_chain(self, k=3):
        """ตั้งค่า QA Chain"""
        try:
            if not self.vectorstore or not self.llm:
                raise ValueError("ต้องมี vectorstore และ llm")

            # สร้าง retriever
            retriever = self.vectorstore.as_retriever(
                search_type="similarity",
                search_kwargs={"k": k}
            )

            # สร้าง custom prompt
            custom_prompt = self.create_custom_prompt()

            # สร้าง QA chain
            self.qa_chain = RetrievalQA.from_chain_type(
                llm=self.llm,
                chain_type="stuff",
                retriever=retriever,
                chain_type_kwargs={"prompt": custom_prompt},
                return_source_documents=True
            )

            logger.info("สร้าง QA Chain สำเร็จ")
            return True

        except Exception as e:
            logger.error(f"เกิดข้อผิดพลาด: {e}")
            return False

    def answer_question(self, question):
        """ตอบคำถาม"""
        try:
            if not self.qa_chain:
                raise ValueError("ไม่มี QA chain ให้ใช้งาน")

            result = self.qa_chain({"query": question})
            answer = result["result"]
            source_docs = result.get("source_documents", [])

            # แปลง source documents
            sources = []
            for doc in source_docs:
                sources.append({
                    "content": doc.page_content,
                    "metadata": doc.metadata
                })

            return {
                "answer": answer,
                "sources": sources,
                "method": "qa"
            }

        except Exception as e:
            return {
                "answer": f"เกิดข้อผิดพลาด: {e}",
                "sources": [],
                "method": "error"
            }

    def initialize(self):
        """เริ่มต้นระบบทั้งหมด"""
        logger.info("เริ่มต้น RAG Service...")

        # โหลด vectorstore
        if not self.load_vectorstore():
            return False

        # ตั้งค่า LLM
        if not self.setup_llm():
            return False

        # ตั้งค่า chains
        if not self.setup_qa_chain():
            return False

        logger.info("RAG Service เริ่มต้นสำเร็จ")
        return True

def main():
    """Main Streamlit application"""

    # Page configuration
    st.set_page_config(
        page_title="RAG Chatbot with LangChain",
        page_icon="🤖",
        layout="wide",
        initial_sidebar_state="expanded"
    )

    # CSS styling
    st.markdown("""
    <style>
    .main-header {
        font-size: 2.5rem;
        color: #1f77b4;
        text-align: center;
        margin-bottom: 2rem;
    }
    </style>
    """, unsafe_allow_html=True)

    # Main header
    st.markdown('<h1 class="main-header">🤖 RAG Chatbot with LangChain + Streamlit</h1>', unsafe_allow_html=True)

    # Sidebar
    with st.sidebar:
        st.header("⚙️ การตั้งค่า")

        # Clear chat button
        if st.button("🗑️ ล้างประวัติการสนทนา", use_container_width=True):
            st.session_state.messages = []
            st.rerun()

    # Initialize RAG Service
    @st.cache_resource
    def get_rag_service():
        """Get cached RAG service"""
        service = RAGService()
        if service.initialize():
            return service
        return None

    rag_service = get_rag_service()

    if rag_service is None:
        st.error("❌ ไม่สามารถเริ่มต้น RAG Service ได้ กรุณาตรวจสอบการตั้งค่า")
        st.info("💡 ตรวจสอบว่า:")
        st.info("- มีไฟล์ .env พร้อม GROQ_API_KEY")
        st.info("- มี Vector Store ในโฟลเดอร์ ./vectorstore")
        return

    # Initialize chat history
    if "messages" not in st.session_state:
        st.session_state.messages = []

    # Display chat history
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

            # Show sources if available
            if message.get("sources"):
                with st.expander("📚 แหล่งข้อมูลอ้างอิง"):
                    for i, source in enumerate(message["sources"], 1):
                        st.markdown(f"**แหล่งที่ {i}:**")
                        st.markdown(f"เนื้อหา: {source['content'][:200]}...")
                        if source.get('metadata'):
                            st.markdown(f"ข้อมูลเพิ่มเติม: {source['metadata']}")
                        st.markdown("---")

    # Chat input
    if prompt := st.chat_input("พิมพ์คำถามของคุณ..."):
        # Add user message to chat history
        st.session_state.messages.append({"role": "user", "content": prompt})

        # Display user message
        with st.chat_message("user"):
            st.markdown(prompt)

        # Generate and display assistant response
        with st.chat_message("assistant"):
            with st.spinner("🤔 กำลังคิดด้วย RAG..."):
                try:
                    response = rag_service.answer_question(prompt)

                    # Display answer
                    st.markdown(response["answer"])

                    # Show sources
                    if response.get("sources"):
                        with st.expander("📚 แหล่งข้อมูลอ้างอิง"):
                            for i, source in enumerate(response["sources"], 1):
                                st.markdown(f"**แหล่งที่ {i}:**")
                                st.markdown(f"เนื้อหา: {source['content'][:200]}...")
                                if source.get('metadata'):
                                    st.markdown(f"ข้อมูลเพิ่มเติม: {source['metadata']}")
                                st.markdown("---")

                    # Add assistant response to chat history
                    st.session_state.messages.append({
                        "role": "assistant", 
                        "content": response["answer"],
                        "sources": response.get("sources", []),
                        "method": response.get("method", "unknown")
                    })

                except Exception as e:
                    error_msg = f"❌ เกิดข้อผิดพลาด: {e}"
                    st.error(error_msg)
                    st.session_state.messages.append({"role": "assistant", "content": error_msg})

    # Quick questions
    st.markdown("---")
    st.subheader("❓ คำถามตัวอย่าง")

    # Categorized quick questions
    categories = {
        "🏛️ ประวัติศาสตร์": [
            "จังหวัดน่านมีประวัติศาสตร์อย่างไร?",
            "จังหวัดน่านเคยเป็นส่วนหนึ่งของอาณาจักรใดบ้าง?"
        ],
        "🏞️ สถานที่ท่องเที่ยว": [
            "จังหวัดน่านมีสถานที่ท่องเที่ยวอะไรบ้าง?",
            "จังหวัดน่านมีวัดสำคัญอะไรบ้าง?"
        ],
        "🍽️ อาหารและวัฒนธรรม": [
            "จังหวัดน่านมีอาหารพื้นเมืองอะไรบ้าง?",
            "จังหวัดน่านมีวัฒนธรรมอะไรที่น่าสนใจ?"
        ],
        "📊 ข้อมูลทั่วไป": [
            "จังหวัดน่านมีประชากรเท่าไหร่?",
            "จังหวัดน่านมีพื้นที่เท่าไหร่?"
        ]
    }

    for category, questions in categories.items():
        with st.expander(category):
            cols = st.columns(2)
            for i, question in enumerate(questions):
                with cols[i % 2]:
                    if st.button(question, use_container_width=True, key=f"quick_{category}_{i}"):
                        # Add user message to chat history
                        st.session_state.messages.append({"role": "user", "content": question})

                        # Generate and add assistant response
                        with st.spinner("🤔 กำลังคิด..."):
                            try:
                                response = rag_service.answer_question(question)

                                # Add assistant response to chat history
                                st.session_state.messages.append({
                                    "role": "assistant", 
                                    "content": response["answer"],
                                    "sources": response.get("sources", []),
                                    "method": response.get("method", "unknown")
                                })

                            except Exception as e:
                                error_msg = f"❌ เกิดข้อผิดพลาด: {e}"
                                st.session_state.messages.append({"role": "assistant", "content": error_msg})

                        st.rerun()

if __name__ == "__main__":
    main()

'''

# เขียนไฟล์ streamlit_app.py
with open("../streamlit_app.py", "w", encoding="utf-8") as f:
    f.write(streamlit_app_code)

print("✅ สร้างไฟล์ streamlit_app.py สำเร็จ")
print("💡 วิธีรัน: streamlit run streamlit_app.py")


✅ สร้างไฟล์ streamlit_app.py สำเร็จ
💡 วิธีรัน: streamlit run streamlit_app.py


## 🚀 วิธีรัน Streamlit App

### คำสั่งสำหรับรัน:

```bash
# รัน Streamlit App
streamlit run streamlit_app.py

# หรือใช้ uv (ตามที่คุณต้องการ)
uv run streamlit run streamlit_app.py
```

### ฟีเจอร์ของ Streamlit App:

1. **Chat Interface** 💬
   - กล่องข้อความสำหรับพิมพ์คำถาม
   - แสดงประวัติการสนทนา
   - แสดงแหล่งอ้างอิง

2. **Sidebar Controls** ⚙️
   - เลือกโหมด RAG (Simple QA / Conversational)
   - ปุ่มล้างประวัติการสนทนา

3. **Quick Questions** ❓
   - คำถามตัวอย่างแบ่งตามหมวดหมู่
   - คลิกเพื่อถามคำถามได้ทันที

4. **Source Citations** 📚
   - แสดงแหล่งข้อมูลอ้างอิง
   - แสดง metadata ของเอกสาร


## 📝 สรุปช่วงที่ 4

### สิ่งที่เราได้เรียนรู้:

1. **Streamlit Framework** 🌐
   - รู้จัก Streamlit และข้อดี
   - Components ที่ใช้ในการสร้าง UI
   - การตั้งค่า page configuration

2. **RAG Service Class** 🏗️
   - สร้าง class ที่รวม RAG functionality
   - โหลด Vector Store และ LLM
   - สร้าง QA และ Conversation chains

3. **Streamlit UI** 🎨
   - สร้าง Chat Interface
   - Sidebar สำหรับการตั้งค่า
   - Quick Questions และ Source Citations

4. **Deployment** 🚀
   - สร้างไฟล์ streamlit_app.py
   - วิธีรัน Streamlit App
   - ใช้ uv สำหรับรัน (ตามที่ต้องการ)

### ไฟล์ที่สร้าง:
- `streamlit_app.py` - Streamlit application หลัก

### ต่อไปในช่วงที่ 5:
เราจะเรียนรู้ Advanced Topics และการปรับปรุงระบบ

---

**⏰ เวลาที่ใช้: 60 นาที**
**📚 ไฟล์ต่อไป: `05_wrapup_advanced_topics.ipynb`**
